# Exercicis del tema 7

In [ ]:
%load_ext asymagic

def mcm_llista(llista):
    """
    Retorna el mínim comú múltipe d'una llista de nombres enters
    """
    if len(llista) == 0:
        return 1
    mcm = llista[0]
    for i in llista[1:]:
        mcm = mcm * i // gcd(mcm, i)
    return mcm

def completa_quadrats(expr,vi,vf):
    """
    Completa quadrats en un polinomi de segon grau
    """
    l = len(vi)
    d2 = [diff(expr,t,2)/2 for t in vi]
    zeros = len([t for t in d2 if t != 0])
    if zeros < len(d2) - 1:
        print("Expressió no vàlida")
        return None, None
    d1 = [diff(expr,t) for t in vi]
    d1 = [d1[i] - 2*d2[i]*vi[i] for i in range(l)]
    ti = expr - sum([d2[i]*vi[i]^2 + d1[i]*vi[i] for i in range(l) if d2[i] != 0])
    c = [d1[i]/(2*d2[i]) if d2[i] != 0 else 0 for i in range(l)]
    if zeros == len(d2):
        s = sum([d2[i]*vf[i]^2 for i in range(l)]) == -ti + sum([d2[i]*c[i]^2 for i in range(l)])
        r = [vf[i] == vi[i] + c[i] for i in range(l)]
        return s, r
    ti = expr - sum([d2[i]*vi[i]^2 + d1[i]*vi[i] for i in range(l)])
    ti -= sum([d2[i]*c[i]^2 for i in range(l)])
    k = d2.index(0)
    s = vf[k] == - sum([d2[i]*vf[i]^2 for i in range(l)])/d1[k]
    r = [vf[i] == vi[i] + c[i] if d2[i] != 0 else vf[i] == vi[i] + ti/d1[k] for i in range(l)]
    return s,r

def centre_conica_quadrica(Q,L):
    """
    Retorna un centre, si és possible, d'una cònica o una quàdrica.
    """
    try:
        C = Q.solve_right(-L)
        return vector(C)
    except:
        return []

def gram_schmidt(vecs):
    """
    Aplica el mètode de Gram-Schmidt a una llista de vectors
    """
    v = []
    for u in vecs:
        if len(v) == 0:
            v.append(simplifica_vector(u))
            continue
        w = u - sum([u.dot_product(x)/x.dot_product(x)*x for x in v])
        v.append(simplifica_vector(w))
    return v
    
def diagonalitza_matriu_simetrica(A):
    """
    Retorna els valors propis i una base ortonormal de vectors propis d'una
    matriu simètrica A
    """
    if A.T != A:
        return None
    D, C = Q.eigenmatrix_right()
    r = D.nrows()
    vaps = [D[i,i] for i in range(r)]
    veps = gram_schmidt([vector(u) for u in C.columns()])
    l = list(zip(vaps,veps))
    p = sorted([x for x in l if x[0] > 0],key=lambda x: x[0])
    n = sorted([x for x in l if x[0] < 0],key=lambda x: -x[0])
    z = [x for x in l if x[0] == 0]
    l = p + n + z
    vaps, veps = map(list,zip(*l))
    if r == 2:
        if veps[0][0] < 0:
            veps[0] = -veps[0]
        if veps[1][1] < 0:
            veps[1] = -veps[1]
    if r == 3:
        if matrix(veps).T.det() < 0:
            veps[2] = -veps[2]
    return vaps, veps

def equacio_reduida_conica_quadrica_amb_centre(Q,L,f,vaps,centre):
    """
    Retorna l'equació reduïda d'una cònica o quàdrica amb centre
    """
    x1, y1, z1 = var('x1 y1 z1')
    X = vector([x1,y1,z1])
    n = Q.nrows()
    q = sum([vaps[i]*X[i]^2 for i in range(n)])
    f1 = centre*Q*centre + 2*L*centre + f
    if f1 != 0:
        eq = q == -f1
        return - eq / f1
    mcm = mcm_llista([x for x in vaps if x != 0])
    eq = q == 0
    return eq / abs(mcm)

def normalitza(l):
    """
    Retorna la llista de vectors normalitzats
    """
    t = list(l)
    return [u.normalized() for u in t]

def simplifica_vector(v):
    """
    Retorna el vector simplificat
    """
    f = []
    for r in v:
        if isinstance(r,Rational):
            f.append(r.denom())
    if len(f) > 0:
        n = prod(f)/gcd(f)
        v = n*v
    f = []
    for r in v:
        f.append(r)
    n = gcd(f)
    return v/n

def coordenades_en_canonica(ref,l):
    """
    Retorna les coordenades en la referència canònica dels punts que
    tenen coordenades "l" en la referència "ref"
    """
    if isinstance(l,list) or isinstance(l,tuple):
        if len(l) == 0:
            return None
        res = []
        for p in l:
            n = len(p)
            res.append(vector(ref[0]) + sum([p[i]*ref[1][i] for i in range(n)]))
        return res
    n = len(l)
    return vector(ref[0]) + sum([l[i]*ref[1][i] for i in range(n)])

def components_en_canonica(base,l):
    """
    Retorna les components en la base canònica del vector que
    té components "l" en la base "base"
    """
    if len(l) == 0:
        return None
    res = []
    for v in l:
        n = len(v)
        res.append(sum([v[i]*base[i] for i in range(n)]))
    return res

def equacio_reduida_parabola(L,vaps,veps):
    """
    Retorna l'equació reduïda d'una paràbola
    """
    x1, y1 = var('x1 y1')
    X = vector([x1,y1])
    v = veps[1]
    v = v.normalized()
    eq = y1 == - vaps[0]*x1^2/(2*v*L)
    return eq

def parametre_parabola(L,vaps,veps):
    """
    Retorna el paràmetre d'una paràbola
    """
    v = veps[1]
    v = v.normalized()
    p = abs(v*L/vaps[0])
    return p

def vertex_parabola(Q,L,f,vaps,veps):
    """
    Retorna el vèrtex d'una paràbola
    """
    x, y = var('x y')
    X = vector([x,y])
    v = veps[0]
    s = vaps[0]*v*X + v*L == 0
    p = X*Q*X + 2*L*X + f == 0
    sol = solve([s,p],[x,y])
    
    return vector([sol[0][0].right(),sol[0][1].right()])

def vertex_paraboloide_cilindre_parabolic(Q,L,f,vaps,veps):
    """
    Retorna el vèrtex d'un paraboloide o els vèrtexs d'un
    cilindre parabòlic
    """
    x, y, z = var('x y z')
    X = vector([x,y,z])
    eqs = []
    for i in range(3):
        if vaps[i] == 0:
            continue
        eq = vaps[i]*veps[i].dot_product(X) + veps[i]*L == 0
        eqs.append(eq)
    eqs.append(X*Q*X + 2*L*X + f == 0)
    sol = solve(eqs,[x,y,z])
    return vector([sol[0][0].right(),sol[0][1].right(),sol[0][2].right()])

def equacio_reduida_quadrica_sense_centre(L,vaps,veps):
    """
    Retorna l'equació reduïda d'una quàdrica sense centre
    """
    x1, y1, z1 = var('x1 y1 z1')
    X = vector([x1,y1,z1])
    v = veps[2]
    v = v.normalized()
    eq = z1 == - vaps[0]*x1^2/(2*v*L) - vaps[1]*y1^2/(2*v*L)
    return eq

def simplificar_radicals(A):
    """
    Simplifica les arrels quadrades en una matriu o en un vector
    """
    try:
        for i in range(A.nrows()):
            for j in range(A.ncols()):
                A[i,j] = A[i,j]._sympy_().simplify()._sage_()
        return A
    except:
        pass
    try:
        for i in range(A.length()):
            A[i] = A[i]._sympy_().simplify()._sage_()
        return A
    except:
        pass
    try:
        return A._sympy_().simplify()._sage_()
    except:
        pass
    if isinstance(A,list):
        try:
            return [simplificar_radicals(x) for x in A]
        except:
            pass
    return None

## Exercici 1
Trobeu l'equació conjunta del parell de rectes $x-2y+2=0$ i $-3x+y-1=0$.

És evident que els punts que compleixen una de les dues equacions han de complir
$$
 (x-2y+2)(-3x+y-1)=0\,,
$$
per tant només cal multiplicar aquests dos termes.

In [ ]:
x, y = var('x y')
eq = expand((x-2*y+2)*(-3*x+y-1) == 0)
show(eq)

L'equació del parell de rectes és
$$
  3x^2 - 7xy + 2y^2 + 7x - 4y + 2 = 0\,.
$$

## Exercici 2
Calculeu l'equació de la circumferència de centre $(2,-3)$ i radi $4$.

Recordem que l'equació de la circumferència de centre $(x_0,y_0)$ i radi $r$ és
$$
  (x-x_0)^2 + (y-y_0)^2 = r^2\,.
$$

In [ ]:
x, y = var('x y')
eq = expand((x-2)^2 + (y+3)^2 - 4^2 == 0)
show(eq)

L'equació de la circumferència és $x^2+y^2-4x+6y-3=0$.

## Exercici 3
Es considera l'el·lipse que té per focus els punts $(2,1)$ i $(0,3)$ i semieix major igual a $2$.
1. Determineu una referència principal i l'equació reduïda.
2. Calculeu la seva equació en la referència canònica.

Els focus i el semieix major són:

In [ ]:
F1 = vector([2,1])
F2 = vector([0,3])
a = 2

La semidistància focal de l'el·lipse és

In [ ]:
c = (F2-F1).norm()/2
show(c)

El semieix menor és

In [ ]:
b = sqrt(a^2-c^2)
show(b)

Per tant, l'equació reduïda de l'el·lipse, és a dir, la seva equació en la referència principal és
$$
  \frac{x'^2}{4}+\frac{y'^2}{2}=1\,.
$$

El centre de la e·lipse és el punt mitjà dels focus:

In [ ]:
B = (F1+F2)/2
show(B)

Les direcció de l'eix principal és

In [ ]:
u1 = F1-B
show(u1)

i, com que $\vec u_2$ ha de ser perpendicular a $\vec u_1$, tenim que $\vec u_2=(1,1)$. Aleshores, la referència principal i l'equació reduïda de l'el·lipse són
$$
  \mathcal{R}'=\left\{(1,2);\frac{1}{\sqrt{2}}(1,-1),\frac{1}{\sqrt{2}}(1,1)\right\}
  \qquad\text{i}\qquad
  \frac{x'^2}{4} + \frac{y'^2}{2} = 1\,.
$$

En el gràfic següent podem veure representada l'el·lipse:

In [ ]:
%%asy --fmt svg
import geometry;
import coordenades;
import graph;

unitsize(1cm);

Canonica(-7,8,-5,9);
Elipse((1,2),(1,-1),4,2,x=8,y=7);

El canvi de coordenades de la referència $\mathcal{R}'$ a la referència canònica vé donat per
$$
  \left(\begin{array}{c} x \\ y \end{array}\right) 
  = \left(\begin{array}{r} 1 \\ 2 \end{array}\right) 
  + \frac{1}{\sqrt{2}}\left(\begin{array}{rr}
   1 & 1 \\
  -1 & 1 \\
  \end{array}\right) 
  \left(\begin{array}{c} x' \\ y' \end{array}\right)\,. 
$$

Per o obtenir l'equació de l'el·lipse en lña referència canònica, hem d'aïllar $(x',y')$ en funció de $(x,y)$ a partir d'aquesta iqualtat i substituir-ho a l'equació reduïda de la el·lipse.

In [ ]:
x , y = var('x y')
x1, y1 = var('x1 y1')
C = 1/sqrt(2)*matrix([[1,1],[-1,1]])
X = vector([x,y])
X1 = vector([x1,y1])
P = C.inverse() * (X - B)
show(P)

In [ ]:
eqr = x1^2/4 + y1^2/2 - 1 == 0
eq = expand(eqr.subs(x1=P[0],y1=P[1]))
## show(eq)
show(8*eq)

L'equació de l'el·lipse en la referència canònica és $3x^2+2xy+3y^2-10x-14y+11 = 0$.

**Una altra manera d'obtenir aquesta equació**

Recodem que si les matrius i el terme independent en la referència canònica d'una cònica són $Q$, $L$ i $f$ i fem el canvi de coordenades
$$
  \left(\begin{array}{c} x\\ y \end{array}\right)
  = B + C\left(\begin{array}{c} x'\\ y'  \end{array}\right)\,,
$$
les matrius i el terme independent de la cònica en la referència $\mathcal{R}'$ són
$$
  \begin{aligned}
  Q' &= C^tQC\\
  L' &= C^t(QB+L)\\
  f' &= B^tQB + 2L^tB + f\,.
  \end{aligned}
$$
Si aïllem $Q$, $L$ i $f$, tindrem que
$$
  \begin{aligned}
  Q &= CQ'C^t\\
  L &= CL' - QB \\
  f &= f' - B^tQB - 2L^tB\,.
  \end{aligned}
$$

Així docs, una altra manera de calcular l'equació de la cònica en la referència és escriure l'eqació reduïda com a
$$
  x'^2 + 2y'^2 - 4 = 0
$$
i aplicar aquestes fórmules.

Escrivim primer les matrius i el terme independent de la cònica en la referència $\mathcal{R}'$:

In [ ]:
Q1 = matrix([[1,0],[0,2]])
L1 = vector([0,0])
f1 = -4 

Calculem ara les matrius i el terme independent de la cònica en la referència canònica

In [ ]:
Q = C*Q1*C.T
L = C*L1 - Q*B
f = f1 - (B*Q*B + 2*L*B)

L'equació de la cònica en la referència canònica és

In [ ]:
eq = expand(X*Q*X + 2*L*X + f) == 0
## show(eq)
show(2*eq)

## Exercici 4
Es considera la paràbola que té per focus el punt $(-2,1)$ i per directriu la recta d'equació
$$x+2y-10=0 \,. $$
1. Determineu una referència principal i l'equació reduïda.
1. Calculeu la seva equació en la referència canònica.

Trobem en primer lloc la projecció ortogonal del focus sobre la recta directriu. La recta perpendicular a la directriu que passa pel focus té equació $2x-y+5=0$. Aleshores,

In [ ]:
x, y = var('x y')
show(solve([x+2*y-10==0,2*x-y+5==0],[x,y]))

In [ ]:
F = vector([-2,1])
F1 = vector([0,5])
V = (F+F1)/2
show(V)

El vèrtex de la paràbola és el punt $(-1,3)$ i les direccions dels eixos principals són $\vec u_1=(2,-1)$ (vector director de la recta directriu) i $\vec u_2=(1,2)$ (perpendicular a l'anterior).

El paràmetre de la paràbola és 

In [ ]:
p = (F-F1).norm()
show(p)

La referència principal de la paràbola és
$$
  \mathcal{R}'=\left\{(-1,3),\frac{1}{\sqrt{5}}(2,-1),\frac{1}{\sqrt{5}}(1,2)\right\}
$$

L'equació reduïda d'una paràbola és de la forma
$$
  y'=\pm\frac{x'^2}{2p}
$$
i per determinar si el signe és positiu o negatiu, hem de veure si el vector $\overrightarrow{VF}$ té el mateix sentit o el contrari que el vector $\vec u_2$:

In [ ]:
show(F-V)

Per tant, l'equació reduïda de la paràbola
$$
   y'=-\frac{x'^2}{4\sqrt{5}}
$$
En el gràfic següent podem veure representada la paràbola:

In [ ]:
%%asy --fmt svg
import geometry;
import coordenades;
import graph;

unitsize(1cm);

Canonica(-12,8,-6,10);
Parabola((-1,3),(-2,1),x=9,y=8);

Calculem l'equació de la paràbola en la referència canònica:

El canvi de coordenades de la referència $\mathcal{R}'$ a la referència canònica vé donat per
$$
  \left(\begin{array}{c} x \\ y \end{array}\right) 
  = \left(\begin{array}{r} -1 \\ 3 \end{array}\right) 
  + \frac{1}{\sqrt{5}}\left(\begin{array}{rr}
   2 & 1 \\
  -1 & 2 \\
  \end{array}\right) 
  \left(\begin{array}{c} x' \\ y' \end{array}\right)\,. 
$$

In [ ]:
x , y = var('x y')
x1, y1 = var('x1 y1')
C = 1/sqrt(5)*matrix([[2,1],[-1,2]])
X = vector([x,y])
X1 = vector([x1,y1])
P = C.inverse() * (X - V)
show(P)

In [ ]:
eqr = y1+x1^2/(4*sqrt(5)) == 0
eq = expand(eqr.subs(x1=P[0],y1=P[1]))
show(eq)

Com que tots els termes estan multiplicats per $\sqrt{5}$ i el denominador comú és 100, multipliquem l'equacio per $\frac{100}{\sqrt{5}}$:

In [ ]:
eq = expand(100/sqrt(5)*eq)
show(eq)

L'equació de la paràbola en la referència canònica és $4x^2-4xy+y^2+40x+30y-75 = 0$.

**Una altra manera d'obtenir l'equació**

Escrivim primer les matrius i el terme independent de la cònica en la referència $\mathcal{R}'$. Per això, observem que l'equació reduïda es pot escriure com
$$
  x'^2 + 4\sqrt{5}\,y' = 0\,.
$$

In [ ]:
Q1 = matrix([[1,0],[0,0]])
L1 = vector([0,2*sqrt(5)])
f1 = 0

Calculem ara les matrius i el terme independent de la cònica en la referència canònica

In [ ]:
Q = C*Q1*C.T
L = C*L1 - Q*B
f = f1 - (B*Q*B + 2*L*B)

L'equació de la cònica en la referència canònica és

In [ ]:
eq = expand(X*Q*X + 2*L*X + f) == 0
## show(eq)
show(5*eq)

## Exercici 5
Es considera la hipèrbola que té per focus els punts $(-1,2)$ i $(3,0)$ i semieix real $2$.
1. Determineu una referència principal i l'equació reduïda.
1. Calculeu la seva equació en la referència canònica.

En primer lloc, escrivim els focus i el semieix real:

In [ ]:
F1 = vector([-1,2])
F2 = vector([3,0])
a = 2

El centre de la hipèrbola ñes el punt mitjà del segment que uneix els focus i la semidistància focal és la meitat de distància entre els focus. Aleshores, també podrem trobar el semieix imaginari.

In [ ]:
B = (F1+F2)/2
show(B)

La semidistància focal és

In [ ]:
c = (F1-F2).norm()/2
show(c)

i el semieix imaginari

In [ ]:
b = sqrt(c^2-a^2)
show(b)

L'equació reduïda de la hipèrbola és
$$
  \frac{x'^2}{4}-y'^2=1
$$
i les direccions dels eixos principals són

In [ ]:
u1 = F2-B
show(u1)

és a dir, $\vec u_1=(2,-1)$ i $\vec u_2=(1,2)$. La referència principal de la hipèrbola és
$$
  \mathcal{R}'=\left\{(1,1),\frac{1}{\sqrt{5}}(2,-1),\frac{1}{\sqrt{5}}(1,2)\right\}\,.
$$
En el gràfic següent podem veure representada la hipèrbola:

In [ ]:
%%asy --fmt svg
import geometry;
import coordenades;
import graph;

unitsize(1cm);

Canonica(-9,12,-8,10);
Hiperbola((1,1),(2,-1),4,1,x=10,y=8);

Calculem l'equació de la paràbola en la referència canònica:

El canvi de coordenades de la referència $\mathcal{R}'$ a la referència canònica vé donat per
$$
  \left(\begin{array}{c} x \\ y \end{array}\right) 
  = \left(\begin{array}{r} 1 \\ 1 \end{array}\right) 
  + \frac{1}{\sqrt{5}}\left(\begin{array}{rr}
   2 & 1 \\
  -1 & 2 \\
  \end{array}\right) 
  \left(\begin{array}{c} x' \\ y' \end{array}\right)\,. 
$$

In [ ]:
x , y = var('x y')
x1, y1 = var('x1 y1')
C = 1/sqrt(5)*matrix([[2,1],[-1,2]])
X = vector([x,y])
X1 = vector([x1,y1])
P = C.inverse() * (X - B)
show(P)

In [ ]:
eqr = x1^2/4-y1^2-1 == 0
eq = expand(eqr.subs(x1=P[0],y1=P[1]))
show(eq)

Com que ens queda el denominador comú 4, podem multiplicar l'equació per 4 o per $-4$:

In [ ]:
eq = -4*eq
show(eq)

L'equació de la hipèrbola en la referència canònica és $4xy+3y^2-4x-10y+11=0$.

**Una altra manera d'obtenir l'equació**

Escrivim primer les matrius i el terme independent de la cònica en la referència $\mathcal{R}'$. Per això, observem que l'equació reduïda es pot escriure com
$$
  x'^2 - 4y'^2 - 4 = 0\,.
$$

In [ ]:
Q1 = matrix([[1,0],[0,-4]])
L1 = vector([0,0])
f1 = -4

Calculem ara les matrius i el terme independent de la cònica en la referència canònica

In [ ]:
Q = C*Q1*C.T
L = C*L1 - Q*B
f = f1 - (B*Q*B + 2*L*B)

L'equació de la cònica en la referència canònica és

In [ ]:
eq = expand(X*Q*X + 2*L*X + f) == 0
## show(eq)
show(-eq)

## Exercici 6
Trobeu les rectes que formen la cònica
$$
  2x^2-2y^2-3xy+4x-3y+2=0 \,.
$$

Es tracta de descomposar en factors el polinomi de segon grau:
$$
  2x^2-2y^2-3xy+4x-3y+2 = (Ax+By+C)(ax+by+c)\,,
$$
i això ho podem fer amb la funció *factor*.

In [ ]:
x, y = var('x y')
eq = 2*x^2-2*y^2-3*x*y+4*x-3*y+2
show(factor(eq))

Les dues rectes tenen equacions $2x+y+2=0$ i $x-2y+1=0$.

## Exercici 7
Determineu el centre i el radi de la circumferència d'equació
$$
  4x^2+4y^2-8x+4y-15=0 \,.
$$

Hem de completar quadrats en aquesta expressió:

In [ ]:
x, y = var('x y')
x1, y1 = var('x1 y1')
eq = 4*x^2+4*y^2-8*x+4*y-15
t, r  = completa_quadrats(eq,[x,y],[x1,y1])
show(t,r)

És a dir, l'equació $4x^2+4y^2-8x+4y-15=0$ es pot escriure com a 
$$
  4(x-1)^2 + 4\left(y+\frac{1}{2}\right)^2 = 20
  \qquad\text{o, dividint per 4}\qquad
  (x-1)^2 + \left(y+\frac{1}{2}\right)^2 = 5\,.
$$
Es tracta, doncs, de la circumferència de centre $C=\left(1,-\frac{1}{2}\right)$ i radi $r=\sqrt{5}$.

## Exercici 8
Es considera la cònica d'equació
$$
   3x^2+2xy+3y^2+16x+16=0\,. 
$$
1. Classifiqueu-la.
2. Determineu els elements geomètrics, els paràmetres i la definició mètrica.
3. Representeu-la gràficament.

Les matrius de la part quadràtica i lineal de la cònica i el terme independent són:
$$
  Q=\left(\begin{array}{rr}
   3 & 1 \\
   1 & 3
  \end{array}\right)\,,\qquad
  L = \left(\begin{array}{r} 8 \\ 0 \end{array}\right)\qquad\text{i}\qquad
  f=16\,.
$$

In [ ]:
Q = matrix([[3,1],[1,3]])
L = vector([8,0])
f = 16

Per calcular el seu centre, si en té, hem de resoldre el sistema d'equacions $QX=-L$, és a dir,
$$
  \left(\begin{array}{rr}
   3 & 1 \\
   1 & 3
  \end{array}\right)
  \left(\begin{array}{r} x \\ y \end{array}\right)
  = \left(\begin{array}{r} -8 \\ 0 \end{array}\right)\,,
$$
i la funció *centre_conica_quadrica* ja ho fa.

In [ ]:
B = centre_conica_quadrica(Q,L)
show(B)

A continuació, necessitem els valors i vectors propis de la matriu $Q$. Recordem del capítol anterior que s'ha de calcular el polinomi característic, els valors propis i, finalment, el vectors propis. La funció *diagonalitza_matriu_simetrica* ja ho fa tot.

In [ ]:
l, v = diagonalitza_matriu_simetrica(Q)
show(l)

Quan els valors propis tenen el mateix signe, és habitual posar en primer lloc el més petit en valor absolut. Aleshores els canviem d'ordre

In [ ]:
show(v)

Els valors propis de $Q$ són $\lambda_1=2$ i $\lambda_2=4$ amb vectors propis $\vec u_1=(1,-1)$ i $\vec u_2=(1,1)$, respectivament.

Ara ja podem calcular l'equació reduïda de la cònica:

In [ ]:
eq = equacio_reduida_conica_quadrica_amb_centre(Q,L,f,l,B)
show(eq)

La cònica és una el·lipse amb referència principal i equació reduïda
$$
  \mathcal{R}'=\left\{(-3,1)\,;\frac{1}{\sqrt{2}}(1,-1),\frac{1}{\sqrt{2}}(1,1)\right\}\qquad\text{i}\qquad
  \frac{x'^2}{4}+\frac{y'^2}{2}=1\,.
$$
El semieix major és $a=2$ i el semieix menor $b=\sqrt{2}$.

En el següent gràfic podem veure l'el·lipse

In [ ]:
%%asy --fmt svg
import geometry;
import coordenades;
import graph;

unitsize(1cm);

Canonica(-9,4,-6,8);
Elipse((-3,1),(1,-1),4,2,x=7,y=6);

La semidistància focal és

In [ ]:
a = 2
b = sqrt(2)
c = sqrt(a^2-b^2)
show(c)

Les coordenades dels focus de l'el·lipse en la referència principal són $(c,0)_{\mathcal{R}'}$ i $(-c,0)_{\mathcal{R}'}$. Calculem ara les seves coordenades en la referència canònica:

In [ ]:
v = normalitza(v)
ref = [B,v]
show(coordenades_en_canonica(ref,[(c,0),(-c,0)]))

Els vèrtexs de l'el·lipse són els punts que en la referència principal tenen coordenades $(a,0)_{\mathcal{R}'}$, $(-a,0)_{\mathcal{R}'}$, $(0,b)_{\mathcal{R}'}$ i $(0,-b)_{\mathcal{R}'}$. Calculem ara les seves coordenades en la referència canònica:

In [ ]:
show(coordenades_en_canonica(ref,[(a,0),(-a,0),(0,b),(0,-b)]))

Els vèrtexs que estan sobre l'eix de les $x'$ són $\big(-3+\sqrt{2},1-\sqrt{2}\big)$ i $\big(-3-\sqrt{2},1+\sqrt{2}\big)$ i els vèrtexs que estan sobre l'eix de les $y'$ són $(-2,2)$ i $(-4,0)$.

**Definició mètrica**: el·lipse formada pels punts del pla tals que la suma de les seves distàncies als punts $(-4,2)$ i $(-2,0)$ és $4$.

## Exercici 9
Es considera la cònica d'equació
$$4x^2-4xy+y^2+32x+34y-11=0 \,. $$
1. Classifiqueu-la.
2. Determineu els elements geomètrics, els paràmetres i la definició mètrica.
3. Representeu-la gràficament.

Essencialment, hem de seguir els mateixos passos que a l'exercici anterior.

Les matrius de la part quadràtica i lineal de la cònica i el terme independent són:
$$
  Q=\left(\begin{array}{rr}
    4 & -2 \\
   -2 &  1
  \end{array}\right)\,,\qquad
  L = \left(\begin{array}{r} 16 \\ 17 \end{array}\right)\qquad\text{i}\qquad
  f=-11\,.
$$

In [ ]:
Q = matrix([[4,-2],[-2,1]])
L = vector([16,17])
f = -11

Centre de la cònica:

In [ ]:
B = centre_conica_quadrica(Q,L)
show(B)

La cònica no té centre, per tant, és una **paràbola**. En aquest cas, haurem de calcular el seu vèrtex.

Valors i vectors propis de la matriu $Q$:

In [ ]:
l, v = diagonalitza_matriu_simetrica(Q)
show(l)

In [ ]:
show(v)

Els vectors propis de $Q$ són $\lambda_1=5$ i $\lambda_2=0$ amb vectors propis $\vec u_1=(2,-1)$ i $\vec u_2=(1,2)$, respectivament.

Calculem ara l'equació reduïda, el paràmetre i el vèrtex de la paràbola, utilitzant les funcions *equacio_reduida_parabola*, *parametre_parabola* i *vertex_parabola*.

In [ ]:
show(equacio_reduida_parabola(L,l,v))

In [ ]:
p = parametre_parabola(L,l,v)
show(p)

In [ ]:
V = vertex_parabola(Q,L,f,l,v)
show(V)

La referència principal i l'equació reduïda de la paràbola és són
$$
  \mathcal{R}'=\left\{(-1,1)\,;\frac{1}{\sqrt{5}}(2,-1),\frac{1}{\sqrt{5}}(1,2)\right\}\qquad\text{i}\qquad
  y'=-\frac{x'^2}{4\sqrt{5}}\,.
$$

El focus de la paràbola és el punt $\left(0,-\sqrt{5}\right)_{\mathcal{R}'}$. Les seves coordenades en la referència canònica són

In [ ]:
v = normalitza(v)
ref = [V,v]
F1 = vector([0,-sqrt(5)])
F = coordenades_en_canonica(ref,F1)
show(F)

El focus és el punt $F=(-2,-1)$. En el gràfic següent podem veure representada la paràbola:

In [ ]:
%%asy --fmt svg
import geometry;
import coordenades;
import graph;

unitsize(1cm);

Canonica(-12,8,-8,8);
Parabola((-1,1),(-2,-1),x=8,y=7);

Finalment, la recta directriu és la recta que té vector director $(2,-1)$ i passa pel punt $F_2$, on

In [ ]:
F2 = vector([0,sqrt(5)])
F = coordenades_en_canonica(ref,F2)
show(F)

és a dir, la seva equació és $x+2y=6$.

**Definició mètrica**: paràbola formada pels punts que equidisten del focus $F=(-2,-1)$ i de la recta directriu $x+2y=6$.

## Exercici 10
Es considera la cònica no degenerada d'equació
$$
  3x^2-4xy-8x+8y=0 \,. 
$$
1. Classifiqueu-la.
2. Determineu els elements geomètrics, els paràmetres i la definició mètrica.

Les matrius de la part quadràtica i lineal de la cònica i el terme independent són:
$$
  Q=\left(\begin{array}{rr}
    3 & -2 \\
   -2 &  0
  \end{array}\right)\,,\qquad
  L = \left(\begin{array}{r} -4 \\ 4 \end{array}\right)\qquad\text{i}\qquad
  f=0\,.
$$

In [ ]:
Q = matrix([[3,-2],[-2,0]])
L = vector([-4,4])
f = 0

Centre de la cònica:

In [ ]:
B = centre_conica_quadrica(Q,L)
show(B)

Valors i vectors propis de la matriu $Q$:

In [ ]:
l, v = diagonalitza_matriu_simetrica(Q)
show(l)

In [ ]:
show(v)

Els valors propis de $Q$ són $\lambda_1=4$ i $\lambda_2=-1$ amb vectors propis $\vec u_1=(2,-1)$ i $\vec u_2=(1,2)$, respectivament.

In [ ]:
eq = equacio_reduida_conica_quadrica_amb_centre(Q,L,f,l,B)
show(eq)

La cònica és una hipèrbola amb referència principal i equació reduïda
$$
  \mathcal{R}'=\left\{(2,1)\,;\frac{1}{\sqrt{5}}(2,-1),\frac{1}{\sqrt{5}}(1,2)\right\}\qquad\text{i}\qquad
  x'^2-\frac{y'^2}{4}=1\,.
$$
El semieix real és $a=1$ i el semieix imaginari $b=2$. La semidistància focal és 

In [ ]:
a = 1
b = 2
c = sqrt(a^2+b^2)
show(c)

En el gràfic següent podem veure la hipèrbola

In [ ]:
%%asy --fmt svg
import geometry;
import coordenades;
import graph;

unitsize(1.3cm);

Canonica(-6,10,-8,9);
Hiperbola((2,1),(2,-1),1,4,x=8,y=8);

Els focus són els punts de coordenades $\left(\sqrt{5},0\right)_{\mathcal{R}'}$ i $\left(-\sqrt{5},0\right)_{\mathcal{R}'}$, mente que és vèrtexs tenen coordenades
$(1,0)_{\mathcal{R}'}$ i $(-1,0)_{\mathcal{R}'}$. Les seves coordenades en la referència canònica són

In [ ]:
B1 = normalitza(v)
ref = [B,B1]
show(coordenades_en_canonica(ref,[(c,0),(-c,0),(a,0),(-a,0)]))

Les components dels vectors directors de les asímtotes (en la base $\mathcal{B}'=\{(2,-1),(1,2)\}$ són $(a,\pm b)_{\mathcal{B}'}$, és a dir, $(1,2)_{\mathcal{B}'}$ i $(1,-2)_{\mathcal{B}'}$. Les components en la base canònica d'aquest vectors són

In [ ]:
B1 = [vector([2,-1]),vector([1,2])]
w1 = vector([a,b])
w2 = vector([a,-b])
w = components_en_canonica(B1,[w1,w2])
show(w)

Per tant, les equacions de les asímptotes de la hipèrbola (recordem que paqssen pel centre $(2,1)$) són $3x-4y=2$ i $x=2$.

**Definició mètrica**: hipèbola formada pels punts tals que el valor absolut de la diferència de les seves distàncies a $(4,0)$ i $(0,2)$ és $2$.

## Exercici 11
Donada la quàdrica d'equació
$$
 7 x^2 + 7 y^2 - 5 z^2 - 34 x y - 10 x z - 10 y z + 136 x - 56 y + 40 z -8 = 0
$$
en la refrència canònica, trobeu la seva equació en la referència $\mathcal{R}'=\left\{(-2,2,4);\,\frac{1}{\sqrt{2}}(1,-1,0),\frac{1}{\sqrt{3}}(1,1,1),\frac{1}{\sqrt{6}}(-1,-1,2)\right\}$.

En aquest cas tenim que les matrius de la part quadràtica i lineal de la cònica i el terme independent són:
$$
  Q=\left(\begin{array}{rrr}
     7 & -17 & -5 \\
   -17 &   7 & -5 \\
    -5 &  -5 & -5
  \end{array}\right)\,,\qquad
  L = \left(\begin{array}{r} 68 \\ -28 \\ 20 \end{array}\right)\qquad\text{i}\qquad
  f=-8\,.
$$
i hem d'aplicar el canvi de coordenades
$$
  \left(\begin{array}{c} x\\ y\\ z \end{array}\right)
  = \left(\begin{array}{r} -2\\ 2\\ 4 \end{array}\right) 
  +\frac{1}{\sqrt{6}}\left(\begin{array}{ccr} 
   \sqrt{3} & \sqrt{2} & -1 \\ 
  -\sqrt{3} & \sqrt{2} & -1 \\ 
   0        & \sqrt{2} &  2
   \end{array}\right) 
  \left(\begin{array}{c} x'\\ y'\\ z' \end{array}\right)\,,
$$

In [ ]:
Q = matrix([[7,-17,-5],[-17,7,-5],[-5,-5,-5]])
L = vector([68,-28,20])
f = -8

In [ ]:
v1 = 1/sqrt(2)*vector([1,-1,0])
v2 = 1/sqrt(3)*vector([1,1,1])
v3 = 1/sqrt(6)*vector([-1,-1,2])
C = matrix([v1,v2,v3]).T
B = vector([-2,2,4])

Sabem que les matrius i el terme independent de l'equació de la quàdrica en la referència $\mathcal{R}'$ són
$$
  \begin{aligned}
  Q' &= C^tQC\\
  L' &= C^t(QB+L)\\
  f' &= B^tQB + 2L^tB + f\,.
  \end{aligned}
$$

In [ ]:
Q1 = C.T*Q*C
L1 = C.T*(Q*B+L)
f1 = B*Q*B +2*L*B + f
show(Q1)

In [ ]:
show(L1)

In [ ]:
show(f1)

In [ ]:
x1, y1, z1 = var('x1 y1 z1')
X1 = vector([x1,y1,z1])
eq = X1*Q1*X1 +2*L1*X1 == -f1
## show(eq)
show(eq/120)

L'equació de la quàdrica en la referència $\mathcal{R}'$ és
$$
  \frac{x'^2}{5}-\frac{y'^2}{8}=1
$$
i es tracta d'un cilindre hiperbòlic.

## Exercici 12
Calculeu l'equació en la referència canònica
de la quàdrica  que en la referència euclidiana $\mathcal{R}'=\left\{(3,-1,2);\,\frac13(2,2,-1),\frac13(2,-1,2),\frac13(-1,2,2)\right\}$ té equació
$$
  x'^2+y'^2-2z'^2=0\,. 
$$

En aquest cas, les matrius $Q'$ i $L'$ i el terme independent $f'$ són
$$
  Q'=\left(\begin{array}{rrr}
   1 & 0 &  0\\
   0 & 1 &  0\\
   0 & 0 & -2
  \end{array}\right)\,,\qquad
  L' = \left(\begin{array}{r} 0 \\ 0 \\ 0 \end{array}\right)\qquad\text{i}\qquad
  f'=0\,.
$$

Recodem que si les matrius i el terme independent en la referència canònica d'una quàdrica són $Q$, $L$ i $f$ i fem el canvi de coordenades
$$
  \left(\begin{array}{c} x\\ y\\ z \end{array}\right)
  = B + C\left(\begin{array}{c} x'\\ y'\\ z' \end{array}\right)\,,
$$
les matrius i el terme indepent de la quàdrica en la referència $\mathcal{R}'$ són
$$
  \begin{aligned}
  Q' &= C^tQC\\
  L' &= C^t(QB+L)\\
  f' &= B^tQB + 2L^tB + f\,.
  \end{aligned}
$$
Si aïllem $Q$, $L$ i $f$, tindrem que
$$
  \begin{aligned}
  Q &= CQ'C^t\\
  L &= CL' - QB \\
  f &= f' - B^tQB - 2L^tB\,.
  \end{aligned}
$$

In [ ]:
Q1 = matrix([[1,0,0],[0,1,0],[0,0,-2]])
L1 = vector([0,0,0])
f1 = 0

In [ ]:
C = 1/3*matrix([[2,2,-1],[2,-1,2],[-1,2,2]])
B = vector([3,-1,2])
show(C)

In [ ]:
Q = C*Q1*C.T
show(Q)

In [ ]:
L = C*L1 - Q*B
show(L)

In [ ]:
f = f1 - (B*Q*B + 2*L*B)
show(f)

L'equació de la quàdrica en la referència canònica és

In [ ]:
x, y, z = var('x y z')
X = vector([x,y,z])
eq = expand(X*Q*X + 2*L*X + f) == 0
## show(eq)
show(3*eq)

és a dir,
$$
  2x^2-y^2-z^2+4xy+4xz-8yz-16x+2y-16z+41=0\,.
$$

## Exercici 13
Trobeu l'equació de la quàdrica formada pel parell de plans
$$
  x-2y+3z-1=0 \qquad \text{i} \qquad y-2z+2=0 \,.
$$

Es tracta d'un parell de plans secants i la seva equació és

In [ ]:
x, y, z = var('x y z')
eq = expand((x-2*y+3*z-1)*(y-2*z+2))
show(eq == 0)

i, si canviem el signe, ens queda l'equació
$$
  2y^2+6z^2-xy+2xz-7yz-2x+5y-8z+2=0\,.
$$

## Exercici 14
Determineu les equacions dels dos plans que formen la quàdrica d'equació
$$
  x^2+3y^2+15z^2+4xy-8xz-14yz=0 \,.
$$

Només cal que descomposem en factors el terme $x^2+3y^2+15z^2+4xy-8xz-14yz$:

In [ ]:
x, y, z = var('x y z')
show(factor(x^2+3*y^2+15*z^2+4*x*y-8*x*z-14*y*z))

Les equacions dels dos plans són $x+3y-5z=0$ i $x+y-3z=0$ i es tracta de dos plans secants.

## Exercici 15
Calculeu l'equació de l'esfera de centre $(1,-2,1)$ i radi $3$.

Recordem que l'equació de l'esfera de centre $(x_0,y_0,z_0)$ i radi $r$ és
$$
  (x-x_0)^2 + (y-y_0)^2 + (z-z_0)^2= r^2\,.
$$

In [ ]:
x, y, z = var('x y z')
eq = expand((x-1)^2+(y+2)^2+(z-1)^2 - 3^2) == 0
show(eq)

L'equació de l'esfera és $x^2 + y^2 + z^2 - 2x + 4y - 2z - 3 = 0$.

## Exercici 16
Determineu el centre i el radi de l'esfera d'equació
$$
  x^2+y^2+z^2-4x+2y-6z+10=0 \,.
$$

Per determinar el centre i el radi, hem de completar quadrats:

In [ ]:
x, y, z = var('x y z')
x1, y1, z1 = var('x1 y1 z1')
eq = x^2 + y^2 + z^2 - 4*x + 2*y - 6*z + 10
show(completa_quadrats(eq,[x,y,z],[x1,y1,z1]))

És a dir, l'equació de l'esfera es pot escriure com a
$$
  (x-2)^2 + (y+1)^2 + (z-3)^3 = 4\,.
$$

Es tracta de la circumferència de centre $C=(2,-1,3)$ i radi $r=2$.

## Exercici 17
Classifiqueu la quàdrica d'equació
$$
  4x^2+4y^2+z^2+2xy-4xz+4yz+2x-2y+4z-6=0 \,.
$$

Les matrius de la part quadràtica i lineal de la cònica i el terme independent són:
$$
  Q=\left(\begin{array}{rrr}
    4 & 1 & -2 \\
    1 & 4 &  2 \\
   -2 & 2 &  1
  \end{array}\right)\,,\qquad
  L = \left(\begin{array}{r} 1 \\ -1 \\ 2 \end{array}\right)\qquad\text{i}\qquad
  f=-6\,.
$$

In [ ]:
Q = matrix([[4,1,-2],[1,4,2],[-2,2,1]])
L = vector([1,-1,2])
f = -6

Centre de la quàdrica: solució de $QX=-L$.

In [ ]:
C = centre_conica_quadrica(Q,L)
show(C)

Valors i vectors propis de la matriu $Q$:

In [ ]:
l, v = diagonalitza_matriu_simetrica(Q)
show(l)

In [ ]:
show(v)

Com que és una quàdrica amb centre, la seva equació reduïda és

In [ ]:
eq = equacio_reduida_conica_quadrica_amb_centre(Q,L,f,l,C)
show(eq)

Per tant, la quàdrica és un **con de revolució**, la seva referència principal és
$$
   \mathcal{R}'=\left\{(1,-1,2);\,\frac{1}{\sqrt{5}}(2,0,-1),\frac{1}{\sqrt{30}}(1,5,2),\frac{1}{\sqrt{6}}(1,-1,2)\right\}
$$
i la seva equació reduïda
$$
  x'^2+y'^2-\frac{z'^2}{5}=0\,.
$$

## Exercici 18
Classifiqueu la quàdrica d'equació
$$
  23 x^2 + 23 y^2 + 8 z^2 - 34 x y + 4 x z + 4 y z + 84 x - 76 y - 24 z + 68 = 0\,.
$$

Les matrius de la part quadràtica i lineal de la cònica i el terme independent són:
$$
  Q=\left(\begin{array}{rrr}
    23 & -17 & 2 \\
   -17 &  23 & 2 \\
     2 &   2 & 8
  \end{array}\right)\,,\qquad
  L = \left(\begin{array}{r} 42 \\ -38 \\ -12 \end{array}\right)\qquad\text{i}\qquad
  f=68\,.
$$

In [ ]:
Q = matrix([[23,-17,2],[-17,23,2],[2,2,8]])
L = vector([42,-38,-12])
f = 68

Centre de la quàdrica:

In [ ]:
B = centre_conica_quadrica(Q,L)
show(B)

Valors i vectors propis de la matriu $Q$:

In [ ]:
l, v = diagonalitza_matriu_simetrica(Q)
show(l)

In [ ]:
show(v)

Equació reduïda:

In [ ]:
eq = equacio_reduida_conica_quadrica_amb_centre(Q,L,f,l,B)
show(eq)

La quàdrica és un **el·lipsoide real**, la seva referència principal és
$$
  \mathcal{R}'=\left\{(-2,0,2);\,\frac{1}{\sqrt{3}}(1,1,-1),\frac{1}{\sqrt{6}}(1,1,2),\frac{1}{\sqrt{2}}(1,-1,0)\right\}
$$
i l'equació reduïda
$$
  \frac{x'^2}{10}+\frac{y'^2}{4}+z'^2=1\,.
$$

## Exercici 19
Classifiqueu la quàdrica d'equació
$$
  37 x^2 + 7 y^2 + 7 z^2 + 34 x y + 34 x z + 26 y z - 154 x - 86 y - 38 z + 13 = 0\,.
$$

Les matrius de la part quadràtica i lineal de la cònica i el terme independent són:
$$
  Q=\left(\begin{array}{rrr}
    37 & 17 & 17 \\
    17 &  7 & 13 \\
    17 & 13 &  7
  \end{array}\right)\,,\qquad
  L = \left(\begin{array}{r} -77 \\ -43 \\ -19 \end{array}\right)\qquad\text{i}\qquad
  f=13\,.
$$

In [ ]:
Q = matrix([[37,17,17],[17,7,13],[17,13,7]])
L = vector([-77,-43,-19])
f = 13

Centre de la quàdrica:

In [ ]:
C = centre_conica_quadrica(Q,L)
show(C)

Valors i vectors propis de $Q$:

In [ ]:
l, v = diagonalitza_matriu_simetrica(Q)
show(l)

In [ ]:
show(v)

Equació reduïda amb aquests valors i vectors propis:

In [ ]:
eq = equacio_reduida_conica_quadrica_amb_centre(Q,L,f,l,C)
show(eq)

In [ ]:
eq = equacio_reduida_conica_quadrica_amb_centre(Q,L,f,l,C)
show(eq)

La quàdrica és un **hiperboloide d'una fulla**, la seva referència principal amb orientació positiva és
$$
  \mathcal{R}'=\left\{(3,-3,1);\,\frac{1}{\sqrt{3}}(1,-1,-1),\frac{1}{\sqrt{6}}(2,1,1),\frac{1}{\sqrt{2}}(0,-1,1)\right\}
$$
i l'equació reduïda
$$
  \frac{x'^2}{36}+\frac{y'^2}{2}-\frac{z'^2}{18}=1\,.
$$

## Exercici 20
Classifiqueu la quàdrica d'equació
$$
  -11x^2 - 11y^2 + 7z^2 + 32xy - 4xz - 4yz - 68x + 40y + 22z + 79 = 0\,.
$$

Les matrius de la part quadràtica i lineal de la cònica i el terme independent són:
$$
  Q=\left(\begin{array}{rrr}
   -11 &  16 &  -2 \\
    16 & -11 &  -2 \\
    -2 &  -2 &   7
  \end{array}\right)\,,\qquad
  L = \left(\begin{array}{r} -34 \\ 20 \\ 11 \end{array}\right)\qquad\text{i}\qquad
  f=79\,.
$$

In [ ]:
Q = matrix([[-11,16,-2],[16,-11,-2],[-2,-2,7]])
L = vector([-34,20,11])
f = 79

Centre de la quàdrica:

In [ ]:
C = centre_conica_quadrica(Q,L)
show(C)

Valors i vectors propis de la matriu $Q$:

In [ ]:
l, v = diagonalitza_matriu_simetrica(Q)
show(l)

In [ ]:
show(v)

In [ ]:
eq = equacio_reduida_conica_quadrica_amb_centre(Q,L,f,l,C)
show(-eq)

Es tracta d'un **hiperboloide de dues fulles**. La referència principal és
$$
  \mathcal{R}'=\left\{(0,2,-1);\,\frac{1}{\sqrt{3}}(1,1,1),\frac{1}{\sqrt{6}}(1,1,-2),\frac{1}{\sqrt{2}}(-1,1,0)\right\}\,,
$$
i l'equació reduïda
$$
  \frac{x'^2}{36}+\frac{y'^2}{12}-\frac{z'^2}{4}=-1\,.
$$

## Exercici 21
Classifiqueu la quàdrica d'equació
$$
  11 x^2 - 5 y^2 + z^2 - 26 x y - 38 x z - 6 y z - 112 x + 28 z -14  = 0\,.
$$

Les matrius de la part quadràtica i lineal de la cònica i el terme independent són:
$$
  Q=\left(\begin{array}{rrr}
    11 & -13 & -19 \\
   -13 &  -5 &  -3 \\
   -19 &  -3 &   1
  \end{array}\right)\,,\qquad
  L = \left(\begin{array}{r} -56 \\ 0 \\ 14 \end{array}\right)\qquad\text{i}\qquad
  f=-14\,.
$$

In [ ]:
Q = matrix([[11,-13,-19],[-13,-5,-3],[-19,-3,1]])
L = vector([-56,0,14])
f = -14

Centre de la quàdrica:

In [ ]:
C = centre_conica_quadrica(Q,L)
show(C)

Valors i vectors propis de la matriu $Q$:

In [ ]:
l, v = diagonalitza_matriu_simetrica(Q)
show(l)

In [ ]:
show(v)

**Observació**: com que hem trobat un centre i la matriu $Q$ té el valor propi $\lambda=0$, sabem que el sistema d'equacions $QX=-L$ és compatible indeterminat.
Trobem la solució general:

In [ ]:
x, y, z = var('x y z')
X = vector([x,y,z])
QX = Q * X
eqs = [QX[i] == -L[i] for i in range(3)]
show(eqs)

In [ ]:
sol = solve(eqs,[y,z])
show(sol)

Aleshores, el punt $C=(0,3,-5)$ també és un centre de la quàdrica.

Equació reduïda

In [ ]:
eq = equacio_reduida_conica_quadrica_amb_centre(Q,L,f,l,C)
show(eq)

Així doncs, la referència principal de la quàdrica és
$$
  \mathcal{R}'=\left\{(0,3,-5);\,\frac{1}{\sqrt{14}}(3,-1,-2),\frac{1}{\sqrt{3}}(1,1,1),\frac{1}{\sqrt{42}}(1,-5,4)\right\}\,,
$$
l'equació reduïda
$$
  \frac{x'^2}{3}-\frac{y'^2}{4}=1
$$
i es tracta d'un **cilindre hiperbòlic**.

## Exercici 22
Classifiqueu la quàdrica d'equació
$$
  5 x^{2} + 2 y^{2} + 2 z^{2} - 2 x y - 2 x z + 4 y z + 26 x + 2 y + 2 z + 11 = 0\,.
$$

Les matrius de la part quadràtica i lineal de la cònica i el terme independent són:
$$
  Q=\left(\begin{array}{rrr}
    5 & -1 & -1 \\
   -1 &  2 &  2 \\
   -1 &  2 &  2
  \end{array}\right)\,,\qquad
  L = \left(\begin{array}{r} 13 \\ 1 \\ 1 \end{array}\right)\qquad\text{i}\qquad
  f=11\,.
$$

In [ ]:
Q = matrix([[5,-1,-1],[-1,2,2],[-1,2,2]])
L = vector([13,1,1])
f = 11

Centre de la quàdrica:

In [ ]:
C = centre_conica_quadrica(Q,L)
show(C)

Valors i vectors propis de la matriu $Q$:

In [ ]:
l, v = diagonalitza_matriu_simetrica(Q)
show(l)

In [ ]:
show(v)

L'equacio reduïda de la quàdrica és

In [ ]:
eq = equacio_reduida_conica_quadrica_amb_centre(Q,L,f,l,C)
show(eq)

Es tracta d'un **cilindre el·líptic**. La referència principal és
$$
  \mathcal{R}'=\left\{(-3,-2,0);\,\frac{1}{\sqrt{3}}(1,1,1),\frac{1}{\sqrt{6}}(2,-1,-1),\frac{1}{\sqrt{2}}(0,1,-1)\right\}\,,
$$
i l'equació reduïda
$$
  \frac{x'^2}{10}+\frac{y'^2}{5}=1\,.
$$

## Exercici 23
Classifiqueu la quàdrica d'equació
$$
  9x^2+2y^2-4z^2-12xy-12xz-2x+24y+14z-5=0 \,.
$$

Les matrius de la part quadràtica i lineal de la cònica i el terme independent són:
$$
  Q=\left(\begin{array}{rrr}
    9 & -6 & -6 \\
   -6 &  2 &  0 \\
   -6 &  0 & -4
  \end{array}\right)\,,\qquad
  L = \left(\begin{array}{r} -1 \\ 12 \\ 7 \end{array}\right)\qquad\text{i}\qquad
  f=-5\,.
$$

In [ ]:
Q = matrix([[9,-6,-6],[-6,2,0],[-6,0,-4]])
L = vector([-1,12,7])
f = -5

Centre de la quàdrica:

In [ ]:
B = centre_conica_quadrica(Q,L)
show(B)

Es tracta d'una quàdrica sense centre. Com que la matriu $Q$ té rang 2, és un paraboloide el·líptic o hiperbòlic.

Valors i vectors propis de la matriu $Q$:

In [ ]:
l, v = diagonalitza_matriu_simetrica(Q)
show(l)

In [ ]:
show(v)

Com que $Q$ té un valor propi positiu i un de negatiu, la quàdrica és un **paraboloide hiperbòlic**.

Càlcul del vèrtex del paraboloide: 

In [ ]:
V = vertex_paraboloide_cilindre_parabolic(Q,L,f,l,v)
show(V)

Per tant, la referència principal (hem canviat de signe el primer vector propi per tal de tenir una base amb orientació positiva) del paraboloide hiperbòlic és
$$
  \mathcal{R}'=\left\{(1,0,1);\,\frac{1}{7}(6,-3,-2),\frac{1}{7}(3,2,6),\frac{1}{7}(-2,-6,3)\right\}\,.
$$

i la seva equació reduïda és

In [ ]:
show(equacio_reduida_quadrica_sense_centre(L,l,v))

és a dir,
$$
  z'=x'^2-\frac{y'^2}{2}\,.
$$

## Exercici 24
Classifiqueu la quàdrica d'equació
$$
  8 x^2 + 29 y^2 + 29 z^2 - 24 x y + 24 x z - 14 y z - 60 x - 86 y + 218 z - 91 = 0\,.
$$

Les matrius de la part quadràtica i lineal de la cònica i el terme independent són:
$$
  Q=\left(\begin{array}{rrr}
     8 & -12 & 12 \\
   -12 &  29 & -7 \\
    12 &  -7 & 29
  \end{array}\right)\,,\qquad
  L = \left(\begin{array}{r} -30 \\ -43 \\ 109 \end{array}\right)\qquad\text{i}\qquad
  f=-91\,.
$$

In [ ]:
Q = matrix([[8,-12,12],[-12,29,-7],[12,-7,29]])
L = vector([-30,-43,109])
f = -91

Centre de la quàdrica:

In [ ]:
B = centre_conica_quadrica(Q,L)
show(B)

Es tracta d'una quàdrica sense centre. Com que la matriu $Q$ té rang 2, és un paraboloide el·líptic o hiperbòlic.

Valors i vectors propis de la matriu $Q$:

In [ ]:
l, v = diagonalitza_matriu_simetrica(Q)
show(l)

Com que els dos valors propis no nuls de $Q$ són positius, la quàdrica és un paraboloide el·líptic.

In [ ]:
show(v)

Vèrtex del paraboloide:

In [ ]:
V = vertex_paraboloide_cilindre_parabolic(Q,L,f,l,v)
show(V)

Equació reduïda:

In [ ]:
eq = equacio_reduida_quadrica_sense_centre(L,l,v)
show(eq)

La quàdrica és un **paraboloide el·líptic**, la seva referència principal amb orientació positiva és
$$
  \mathcal{R}'=\left\{(-3,-1,-2);\,\frac{1}{\sqrt{3}}(0,1,1),\frac{1}{\sqrt{22}}(2,-3,3),\frac{1}{\sqrt{11}}(3,1,-1)\right\}
$$
i l'equació reduïda
$$
  z'=\frac{x'^2}{2\sqrt{11}}+\frac{y'^2}{\sqrt{11}}\,.
$$

## Exercici 25
Classifiqueu la quàdrica d'equació
$$
  x^2+y^2+4z^2+2xy+4xz+4yz-48x-24y+84=0 \,.
$$

Les matrius de la part quadràtica i lineal de la cònica i el terme independent són:
$$
  Q=\left(\begin{array}{rrr}
    1 & 1 & 2 \\
    1 & 1 & 2 \\
    2 & 2 & 4
  \end{array}\right)\,,\qquad
  L = \left(\begin{array}{r} -24 \\ -12 \\ 0 \end{array}\right)\qquad\text{i}\qquad
  f=84\,.
$$

In [ ]:
Q = matrix([[1,1,2],[1,1,2],[2,2,4]])
L = vector([-24,-12,0])
f = 84

Centre de la quàdrica:

In [ ]:
B = centre_conica_quadrica(Q,L)
show(B)

Com que la quàdrica no té centre i la matriu $Q$ té rang 1, podem assegurar que es tracta d'un **cilindre parabòlic**.

Valors i vectors propis de la matriu $Q$:

In [ ]:
l, v = diagonalitza_matriu_simetrica(Q)
show(l)

In [ ]:
show(v)

Aquests no són necessàriament els vectors que hem d'escollir per a la referència principal. La referència principal estarà formada pels vectors

In [ ]:
v1 = simplifica_vector(v[0])
v2 = simplifica_vector(v1.cross_product(L))
v3 = simplifica_vector(v1.cross_product(v2))
v = [v1,v2,v3]
show(v)

In [ ]:
V = vertex_paraboloide_cilindre_parabolic(Q,L,f,l,v)
show(V)

D'entre tots els punts de la recta de vèrtexs del cilindre parabòlic, n'escollim un com a origen de la referència principal. Si donem al paràmetre el valor $r_1=1$, tindrem el punt $V=(1,3,1)$.

Aleshores, un punt de la recta de vèrtexs del cilindre parabòlic és el punt $V=(1,3,1)$, la referència principal (recordem que no és única) és
$$
  \mathcal{R}'=\left\{(1,3,1);\,\frac{1}{\sqrt{6}}(1,1,2),\frac{1}{\sqrt{21}}(2,-4,1),\frac{1}{\sqrt{14}}(3,1,-2)\right\}
$$

i l'equació reduïda

In [ ]:
eq = equacio_reduida_quadrica_sense_centre(L,l,v)
show(eq)

és a dir,
$$
  z'=\frac{x'^2}{2\sqrt{14}}\,.
$$